## Initial setup

**Folders need to be created before running the notebook: mNP-best_classify_models and mNP-classify_scores**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
print(tf.__version__)
import torch
print(torch.__version__)
import matplotlib
print(matplotlib.__version__)

2.8.0
1.11.0+cu113
3.2.2


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# Other imports
! pip install tensorflow_addons
! pip install tensorflow_io

import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
from imutils import paths
from tqdm import tqdm
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import tensorflow_io as tfio
import tensorflow_hub as hub
import numpy as np
import cv2
import pandas as pd
import seaborn as sns
from scipy.stats import mannwhitneyu
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import sklearn.manifold
from sklearn.metrics.pairwise import cosine_similarity as cos
from sympy.utilities.iterables import multiset_permutations
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
from IPython.display import Image, display


import zipfile
import concurrent.futures

# Random seed fix
random_seed = 42
tf.random.set_seed(random_seed)
np.random.seed(random_seed)

     |████████████████████████████████| 1.1 MB 14.0 MB/s 
     |████████████████████████████████| 25.9 MB 1.2 MB/s 
     |████████████████████████████████| 2.4 MB 53.5 MB/s 
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.25.0
    Uninstalling tensorflow-io-gcs-filesystem-0.25.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


## Dataset gathering and preparation

In [ ]:
training_batch_size = 4

BATCH_SIZE = training_batch_size

imageSize = 224

category_names = ['bundle', 'dispersed', 'network', 'singular']
color_method = ['C0', 'C1', 'C2', 'C3', 'C4']
color = ['black', 'magenta', 'cyan', 'yellow']
marker = ['o', 's', '<', '>', '^']
seaborn_palette = sns.color_palette("colorblind")

In [ ]:
np.random.seed(random_seed)
peptide_morph_train_path = "/content/drive/MyDrive/TEM image datasets/2022-AutoDetect-mNP-morphology"
peptide_morph_images_train = list(paths.list_files(basePath=peptide_morph_train_path, validExts='jpg'))
peptide_morph_images_train = np.random.choice(np.array(peptide_morph_images_train), len(peptide_morph_images_train), replace=False)
print(len(peptide_morph_images_train))

300


In [ ]:
train_labels = []
for i in range(peptide_morph_images_train.shape[0]):
  train_label = peptide_morph_images_train[i].split("/")[-2]
  train_labels.append(train_label)
le = LabelEncoder()
peptide_morph_train_enc = le.fit_transform(train_labels)

In [ ]:
# Image preprocessing utils
@tf.function
def parse_images(image_path):
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    # image = tfio.experimental.image.decode_tiff(image_string)[:, :, :3]   # in the doc, it transforms tiff to 4 channels, with additional channel of opacity which is not needed.
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[imageSize, imageSize])

    return image

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(peptide_morph_images_train)
train_ds = (
    train_ds
    .map(parse_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # .shuffle(200)
    .batch(training_batch_size
          #  , drop_remainder=True
           )
    .prefetch(tf.data.experimental.AUTOTUNE)
)

datagen = tf. keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

## Initiate self-supervised models

In [ ]:
Resnet50_transfer = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(imageSize, imageSize, 3), 
    pooling=None,
)

Resnet50_transfer.trainable = False

94781440/94765736 [==============================] - 0s 0us/step


In [ ]:
# Resnet as backbone
def get_resnet_self_supervise_model(hidden_1, hidden_2, hidden_3):
    base_model = Resnet50_transfer
    base_model.trainable = True
    inputs = Input((imageSize, imageSize, 3))
    h = base_model(inputs, training=True)
    h = GlobalAveragePooling2D()(h)

    projection_1 = Dense(hidden_1)(h)                                        
    projection_1 = Activation("relu")(projection_1)
    projection_1 = BatchNormalization(epsilon=0.001)(projection_1)
    projection_2 = Dense(hidden_2)(projection_1)
    projection_2 = Activation("relu")(projection_2)
    projection_2 = BatchNormalization(epsilon=0.001)(projection_2)
    projection_3 = Dense(hidden_3)(projection_2)
    projection_3 = BatchNormalization(epsilon=0.001)(projection_3)

    resnet_model = Model(inputs, projection_3)
    
    return resnet_model

In [ ]:
# state-of-the-arts simclr encoder trained on imagenet
saved_model_path = 'gs://simclr-checkpoints-tf2/simclrv2/pretrained/r50_1x_sk0/saved_model/'
imagenet_simclr_model = tf.saved_model.load(saved_model_path)

for image_batch in train_ds.take(1):
  simclr_logits = imagenet_simclr_model(image_batch, trainable=False)

simclr_layer_shape = simclr_logits['final_avg_pool'].shape[1]

# obtain feature maps from the state-of-the-arts simclr encoder trained on imagenet
imagenet_simclr_feature_map = np.zeros((len(peptide_morph_images_train), simclr_layer_shape))

counter = 0
for train_image_batch in train_ds:
  imagenet_simclr_feature_map[counter * training_batch_size: (counter + 1) * training_batch_size] = imagenet_simclr_model(train_image_batch, trainable=False)['final_avg_pool']
  counter += 1

## Initiate downstream classification model

In [ ]:
def get_linear_model(features):                                                                                  
    linear_model = Sequential([                                                                                  
			                              Input(shape=(features,)),
		                                Dense(3, activation="softmax")])
    return linear_model

## Performance assessment of label-efficient training of downstream classification task in main manuscript

In [ ]:
# Random seed fix
random_seed_list = np.array([42, 43, 44, 45, 46])
random_seed_for_split = np.linspace(42, 42 + 19, 20).astype(int)
training_image_size = np.array([80, 32, 16, 8, 4, 2, 1])

# list of models
models = ['barlow_tem', 'simclr_tem', 'barlow_ImageNet', 'simclr_ImageNet', 'simclr_ImageNet_sota']


earlystop_criterion = EarlyStopping(monitor='val_accuracy', patience=10, verbose=0, mode='auto', restore_best_weights=True)
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
metrics = ['accuracy']

In [ ]:
# for self-supervised models trained on tem images or ImageNet images (832 images)
for m in range(0, 4):
  linear_scores = np.zeros((len(training_image_size), len(random_seed_list), len(random_seed_for_split), 4))
  fusion_matrix = np.zeros((len(training_image_size), len(random_seed_list), len(random_seed_for_split), 3, 3))
    
  for i in range(len(random_seed_list)):
    resnet_model = get_resnet_self_supervise_model(128, 64, 1024)
    if models[m] == 'barlow_tem':
      resnet_model.load_weights('barlow_resnet_batch64_project128_64_1024_seed%i.h5' % (random_seed_list[i]))
    if models[m] == 'simclr_tem':
      resnet_model.load_weights('simclr_resnet_batch64_project128_64_1024_nocrop_seed%i.h5' % (random_seed_list[i]))
    if models[m] == 'barlow_ImageNet':
      resnet_model.load_weights('%s_batch64_project128_64_1024_seed%i.h5' % (models[m], random_seed_list[i]))
    if models[m] == 'simclr_ImageNet':
      resnet_model.load_weights('%s_batch64_project128_64_1024_nocrop_seed%i.h5' % (models[m], random_seed_list[i]))


    resnet_model.layers[1].trainable = False

    feature_extraction_model = Model(resnet_model.input, resnet_model.layers[-9].output)

    # Extract train and test features
    features = feature_extraction_model.predict(train_ds)

    for j in range(len(random_seed_for_split)):
      train_feature, test_feature, train_label, test_label = train_test_split(features, peptide_morph_train_enc, test_size=0.2, shuffle=True, stratify=peptide_morph_train_enc, random_state=random_seed_for_split[j])
      train_filename, test_filename, train_label, test_label = train_test_split(peptide_morph_images_train, peptide_morph_train_enc, test_size=0.2, shuffle=True, stratify=peptide_morph_train_enc, random_state=random_seed_for_split[j])      
      train_label_storage = pd.DataFrame(np.concatenate((train_filename.reshape(1, -1).transpose(), train_label.reshape(1, -1).transpose()), axis=-1), columns=['filename', 'label'])
      test_label_storage = pd.DataFrame(np.concatenate((test_filename.reshape(1, -1).transpose(), test_label.reshape(1, -1).transpose()), axis=-1), columns=['filename', 'label'])
    
      for n in range(len(training_image_size)):
        train_images_directory = train_label_storage.groupby('label').sample(n=training_image_size[n], random_state = 42).sample(frac=1, random_state = 42)['filename'].to_numpy().astype(str)
        train_images_label = train_label_storage.groupby('label').sample(n=training_image_size[n], random_state = 42).sample(frac=1, random_state = 42)['label'].to_numpy().astype(int)
        train_dataframe = pd.DataFrame(np.concatenate((train_images_directory.reshape(1, -1).transpose(), train_images_label.reshape(1, -1).transpose()), axis=-1), columns=['filename', 'label'])
      
        checkpoint_model_linear = ModelCheckpoint('mNP-best_classify_models/%s_%iper_class_seed%i_seed%i_linear.h5' 
                                      % (models[m], training_image_size[n], random_seed_list[i], random_seed_for_split[j]),
                                      monitor='val_accuracy', mode='auto', verbose=0, save_best_only=True, save_weights_only=True)
        train_feature_map = train_feature[train_label_storage.groupby('label').sample(training_image_size[n], random_state = 42).sample(frac=1, random_state = 42).index.to_numpy()]
        # train linear classifier model
        linear_model = get_linear_model(train_feature.shape[1])
        linear_model.compile(loss="sparse_categorical_crossentropy", metrics=metrics, optimizer=adam)
        linear_history = linear_model.fit(train_feature_map, train_images_label, validation_data=(test_feature, test_label), 
                                              batch_size=training_batch_size, epochs=300, workers=8, use_multiprocessing=True, 
                                              verbose=1, callbacks=[earlystop_criterion, checkpoint_model_linear])
        # log best classification model performance
        linear_model.load_weights('mNP-best_classify_models/%s_%iper_class_seed%i_seed%i_linear.h5' 
                                    % (models[m], training_image_size[n], random_seed_list[i], random_seed_for_split[j]))
        y_pred_linear = np.argmax(linear_model.predict(test_feature), axis=-1)

        linear_scores[n, i, j] = np.array([accuracy_score(y_pred_linear, test_label), 
                                                precision_score(y_pred_linear, test_label, average='weighted'), 
                                                recall_score(y_pred_linear, test_label, average='weighted'),
                                                f1_score(y_pred_linear, test_label, average='weighted')])
        fusion_matrix[n, i, j] = confusion_matrix(y_pred_linear, test_label)
    
          
  np.savez_compressed('mNP-classify_scores/%s_classification_result.npz' %(models[m]), scores=linear_scores, fusion_matrix=fusion_matrix)

In [ ]:
# for sota self-supervised encoder obtained from SimCLR official implementation

linear_scores = np.zeros((len(training_image_size), len(random_seed_for_split), 4))
fusion_matrix = np.zeros((len(training_image_size), len(random_seed_for_split), 3, 3))

for j in range(len(random_seed_for_split)):

  train_feature, test_feature, train_label, test_label = train_test_split(imagenet_simclr_feature_map, peptide_morph_train_enc, test_size=0.2, shuffle=True, stratify=peptide_morph_train_enc, random_state=random_seed_for_split[j])
  
  # reason why split is used twice: filename, feature_map and labels all need to apply train_test_split, cannot do it in one command, but since they were originally in sync
  # they can be done in two commands without issue

  train_filename, test_filename, train_label, test_label = train_test_split(peptide_morph_images_train, peptide_morph_train_enc, test_size=0.2, shuffle=True, stratify=peptide_morph_train_enc, random_state=random_seed_for_split[j])      
  train_label_storage = pd.DataFrame(np.concatenate((train_filename.reshape(1, -1).transpose(), train_label.reshape(1, -1).transpose()), axis=-1), columns=['filename', 'label'])
  test_label_storage = pd.DataFrame(np.concatenate((test_filename.reshape(1, -1).transpose(), test_label.reshape(1, -1).transpose()), axis=-1), columns=['filename', 'label'])

  for n in range(len(training_image_size)):
    train_images_directory = train_label_storage.groupby('label').sample(n=training_image_size[n], random_state = 42).sample(frac=1, random_state = 42)['filename'].to_numpy().astype(str)
    train_images_label = train_label_storage.groupby('label').sample(n=training_image_size[n], random_state = 42).sample(frac=1, random_state = 42)['label'].to_numpy().astype(int)
    train_dataframe = pd.DataFrame(np.concatenate((train_images_directory.reshape(1, -1).transpose(), train_images_label.reshape(1, -1).transpose()), axis=-1), columns=['filename', 'label'])
    
    checkpoint_model_linear = ModelCheckpoint('mNP-best_classify_models/%s_%iper_class_seed%i_linear.h5' 
                                  % (models[-1], training_image_size[n], random_seed_for_split[j]),
                                  monitor='val_accuracy', mode='auto', verbose=0, save_best_only=True, save_weights_only=True)   

    train_feature_map = train_feature[train_label_storage.groupby('label').sample(training_image_size[n], random_state = 42).sample(frac=1, random_state = 42).index.to_numpy()]
    # train linear classifier model
    linear_model = get_linear_model(train_feature.shape[1])
    linear_model.compile(loss="sparse_categorical_crossentropy", metrics=metrics, optimizer=adam)
    linear_history = linear_model.fit(train_feature_map, train_images_label, validation_data=(test_feature, test_label), 
                                          batch_size=training_batch_size, epochs=300, workers=8, use_multiprocessing=True, 
                                          verbose=1, callbacks=[earlystop_criterion, checkpoint_model_linear])
    # log best classification model performance
    linear_model.load_weights('mNP-best_classify_models/%s_%iper_class_seed%i_linear.h5' 
                                % (models[-1], training_image_size[n], random_seed_for_split[j]))
    y_pred_linear = np.argmax(linear_model.predict(test_feature), axis=-1)

    linear_scores[n, j] = np.array([accuracy_score(y_pred_linear, test_label), 
                                            precision_score(y_pred_linear, test_label, average='weighted'), 
                                            recall_score(y_pred_linear, test_label, average='weighted'),
                                            f1_score(y_pred_linear, test_label, average='weighted')])

  np.savez_compressed('mNP-classify_scores/%s_classification_result.npz' %(models[-1]), scores=linear_scores, fusion_matrix=fusion_matrix)

Epoch 1/300
60/60 [==============================] - 28s 455ms/step - loss: 0.7615 - accuracy: 0.6667 - val_loss: 0.5837 - val_accuracy: 0.7667
Epoch 2/300
60/60 [==============================] - 0s 3ms/step - loss: 0.4464 - accuracy: 0.8792 - val_loss: 0.4759 - val_accuracy: 0.7667
Epoch 3/300
60/60 [==============================] - 0s 3ms/step - loss: 0.3482 - accuracy: 0.9042 - val_loss: 0.4121 - val_accuracy: 0.8167
Epoch 4/300
60/60 [==============================] - 0s 3ms/step - loss: 0.2798 - accuracy: 0.9375 - val_loss: 0.3821 - val_accuracy: 0.8833
Epoch 5/300
60/60 [==============================] - 0s 3ms/step - loss: 0.2411 - accuracy: 0.9458 - val_loss: 0.3688 - val_accuracy: 0.9000
Epoch 6/300
60/60 [==============================] - 0s 3ms/step - loss: 0.2100 - accuracy: 0.9625 - val_loss: 0.3488 - val_accuracy: 0.8833
Epoch 7/300
60/60 [==============================] - 0s 3ms/step - loss: 0.1907 - accuracy: 0.9708 - val_loss: 0.3424 - val_accuracy: 0.8667
Epoch 8/30

Epoch 1/300
2/2 [==============================] - 1s 210ms/step - loss: 1.2498 - accuracy: 0.0000e+00 - val_loss: 0.9861 - val_accuracy: 0.7667
Epoch 2/300
2/2 [==============================] - 0s 46ms/step - loss: 0.9708 - accuracy: 0.8333 - val_loss: 0.8769 - val_accuracy: 0.7333
Epoch 3/300
2/2 [==============================] - 0s 80ms/step - loss: 0.6580 - accuracy: 0.8333 - val_loss: 0.8124 - val_accuracy: 0.7333
Epoch 4/300
2/2 [==============================] - 0s 40ms/step - loss: 0.5248 - accuracy: 0.8333 - val_loss: 0.7970 - val_accuracy: 0.6667
Epoch 5/300
2/2 [==============================] - 0s 39ms/step - loss: 0.3809 - accuracy: 1.0000 - val_loss: 0.7474 - val_accuracy: 0.7000
Epoch 6/300
2/2 [==============================] - 0s 39ms/step - loss: 0.2766 - accuracy: 1.0000 - val_loss: 0.7119 - val_accuracy: 0.7167
Epoch 7/300
2/2 [==============================] - 0s 38ms/step - loss: 0.2046 - accuracy: 1.0000 - val_loss: 0.6740 - val_accuracy: 0.7500
Epoch 8/300
2/2

Streaming output truncated to the last 5000 lines.
3/3 [==============================] - 0s 50ms/step - loss: 0.0948 - accuracy: 1.0000 - val_loss: 0.3921 - val_accuracy: 0.8333
Epoch 8/300
3/3 [==============================] - 0s 30ms/step - loss: 0.0756 - accuracy: 1.0000 - val_loss: 0.3802 - val_accuracy: 0.8333
Epoch 9/300
3/3 [==============================] - 0s 60ms/step - loss: 0.0622 - accuracy: 1.0000 - val_loss: 0.3726 - val_accuracy: 0.8500
Epoch 10/300
3/3 [==============================] - 0s 43ms/step - loss: 0.0510 - accuracy: 1.0000 - val_loss: 0.3692 - val_accuracy: 0.8500
Epoch 11/300
3/3 [==============================] - 0s 46ms/step - loss: 0.0439 - accuracy: 1.0000 - val_loss: 0.3687 - val_accuracy: 0.8500
Epoch 12/300
3/3 [==============================] - 0s 44ms/step - loss: 0.0384 - accuracy: 1.0000 - val_loss: 0.3692 - val_accuracy: 0.8500
Epoch 13/300
3/3 [==============================] - 0s 44ms/step - loss: 0.0347 - accuracy: 1.0000 - val_loss: 0.3689 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Streaming output truncated to the last 5000 lines.
12/12 [==============================] - 0s 11ms/step - loss: 0.0517 - accuracy: 1.0000 - val_loss: 0.2898 - val_accuracy: 0.9000
Epoch 1/300
6/6 [==============================] - 0s 32ms/step - loss: 0.8869 - accuracy: 0.6250 - val_loss: 0.5914 - val_accuracy: 0.7667
Epoch 2/300
6/6 [==============================] - 0s 18ms/step - loss: 0.5374 - accuracy: 0.8333 - val_loss: 0.4986 - val_accuracy: 0.8000
Epoch 3/300
6/6 [==============================] - 0s 21ms/step - loss: 0.2891 - accuracy: 0.9167 - val_loss: 0.3911 - val_accuracy: 0.8167
Epoch 4/300
6/6 [==============================] - 0s 21ms/step - loss: 0.2049 - accuracy: 0.8750 - val_loss: 0.3524 - val_accuracy: 0.8667
Epoch 5/300
6/6 [==============================] - 0s 20ms/step - loss: 0.1495 - accuracy: 0.9583 - val_loss: 0.3064 - val_accuracy: 0.9000
Epoch 6/300
6/6 [==============================] - 0s 19ms/step - loss: 0.1150 - accuracy: 1.0000 - val_loss: 0.2990 - 